# Sales forecasting
### ... with a lot less historical data than we would prefer


This notebook contains my solution to a hypothetical business problem relating to a company's sales data, the source of which is a truncated version of a data set I downloaded off Kaggle. The data is a few years old now, so in the context of this task it is currently early 2020 and we have no reason to suspect that the remainder of the year will be notably dissimilar to the last couple of years.

In addition to this notebook file, there are two other files in the directory:
- sales.csv (The historical sales data, provided in monthly intervals)
- requirements.txt (Python dependencies)

### Table of contents
* [Business Problem](#business-problem)
* [The Solution](#solution)
    * [Install dependencies](#install-dependencies)
    * [Import modules](#import-modules)
    * [Characteristics of the time series](#characteristics-time-series)
    * [Model building](#model-building)
        * [Random Walk](#random-walk)
        * [ETS](#ets)
        * [ARIMA](#arima)
        * [Regression](#regression)
    * [Comparing each model's 12-month forecast](#comparing-model-forecasts)
    * [Model cross validation](#model-cross-validation)
    * [Evaluation of the models](#evaluation-models)
* [Conclusions](#conclusions)

## <a id="business-problem">Business Problem</a>
***

A retail business launched a new product in the final quarter of 2017, which has historically been their most profitable time of the year due to increased consumer spending leading up to Christmas. The new product has been selling well and appears to have been gaining momentum in the market over the past 2 years. The business would now like to know how well the product is expected to sell over the coming 12 months.

The product was incrementally soft launched over a number of weeks and as a result stable data is only available from January 2018 up to February 2020 (last month).

## <a id="solution">The Solution</a>
***
### <a id="install-dependencies">Install dependencies</a>

In [ ]:
!pip install -r requirements.txt

### <a id="import-modules">Import modules</a>

In [151]:
import altair as alt
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from statsforecast.models import SeasonalNaive
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from pmdarima.arima import auto_arima

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Disable scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Altair custom theme
@alt.theme.register('theme', enable = True)
def theme():
    return alt.theme.ThemeConfig({
        'background': '#FFFFFF',
        'config': {
            'axis': {'domainOpacity': 0, 'gridColor': '#ECECEC', 'labelFontSize': 10, 'labelPadding': 18, 'tickColor': '#ECECEC', 'tickSize': 8},
            'legend': {'clipHeight': 16, 'symbolStrokeWidth': 10, 'labelFontSize': 12},
            'line': {'size': 4},
            'mark': {'color': '#000000'},
            'point': {'fill': '#BABABA', 'size': 150},
            'range': {'category': {'scheme': 'category10'}}
        },
        'height': 400,
        'width': 800
    })


### <a id="characteristics-time-series">Characteristics of the time series</a>

The time series contains aggregated monthly sales over a 26-month period from Jan '18 through to Feb '20. The summary statistics and figure displaying the sales over time reveal a number of characteristics:
- There have been an average of 450,000 sales per month.
- The standard deviation is 240,000 sales per month, while the IQR is only 150,000; indicating the distribution skews to the right.
- There was a sharp increase in sales during November and December in both years, likely attributable to the Christmas spending mentioned above.
- There appears to be a slightly upward trend in sales over time.

In [152]:
ts = pd.read_csv('sales.csv')
ts['Date'] = pd.to_datetime(ts['Date'])
ts.set_index('Date', inplace = True)

ts.describe().T


,count,mean,std,min,25%,50%,75%,max
Sales,26.000,449354.231,238314.771,236985.000,310489.000,368518.000,461642.500,1144687.000


In [166]:
# Calculate trend line 
trend_index = np.arange(0, len(ts)).reshape(-1, 1)
lm = LinearRegression().fit(trend_index, ts['Sales'].to_numpy())
trend_line = pd.DataFrame({'Date': ts.reset_index()['Date'], 'Sales': lm.predict(trend_index)})

alt.Chart(ts.reset_index()).mark_line().encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = 'Sales'
) + alt.Chart(trend_line).mark_line(color = '#F4A460').encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = 'Sales'
).properties(
    title = "Monthly sales from Jan '18 to Feb '20 with linear trend"
)


alt.LayerChart(...)

### <a id="model-building">Model building</a>

Due to the characteristics of the time series we will test out a few different models to find the best fit. The proposed models are:
- Random Walk
- ETS (Error Trend Seasonality)
- ARIMA (Autoregressive Integrated Moving Average)
- Regression

#### <a id="random-walk">Random Walk</a>

A monthly random walk model is a good place to start; each forecast will be equal to the number of sales in the same month of the previous year.


In [154]:
random_walk_mdl = SeasonalNaive(season_length = 12)
random_walk_mdl.fit(ts['Sales'])
random_walk_pred = random_walk_mdl.predict(h = 12)['mean']


#### <a id="ets">ETS</a>

The upward trend and seasonal pattern make ETS an appealing option. The end of year spike in 2019 looked sharper than the previous year, so the seasonal component was modelled as multiplicative in an attempt to capture the increasing variance.


In [130]:
ets_mdl = ETSModel(ts['Sales'], trend = 'add', seasonal = 'mul', freq = 'ME')
ets_mdl_fit = ets_mdl.fit(disp = False)
ets_pred = ets_mdl_fit.forecast(steps = 12)


#### <a id="arima">ARIMA</a>

If we're building an ETS model then we may as well also include an ARIMA model. To help the automatic model selection process handle the small sample size, we increased the traditional alpha level of 0.05 up to a more casual 0.2. There isn't enough data to fit a seasonal component.

The best fitting model selected was (0, 2, 3), which is conceptually similar to a moving average.

In [131]:
arima_mdl = auto_arima(ts['Sales'], stepwise = False, alpha = 0.2)
arima_pred = arima_mdl.predict(12)
arima_mdl

ARIMA(order=(0, 2, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True, with_intercept=False)

#### <a id="regression">Regression</a>

With some minor alterations to our data we can also fit a regression model. We need to expand the data into a tabular format and use the months and trend as features to build the model. The monthly sales have been fit using a logarithmic scale to reduce the impact of the distribution's skewing on the forecasted values.


In [143]:
def build_features_from_date_series(s, trend_offset = 0):

    s.name = 'Date'

    # Generate dummy dates in case the data passed into the function doesn't contain a full 12 months
    # All 12 months need to be created as columns
    dummy_dates = pd.Series(pd.date_range(start = '1900-01-31', freq = 'ME', periods = 12), name = 'Date')
    
    df = pd.DataFrame(pd.concat([s, dummy_dates], ignore_index = True))

    # Encode month names as dummy variables
    df['Month'] = df['Date'].dt.month_name()
    df = pd.get_dummies(df, columns = ['Month'], dtype = 'int')

    # Use incrementing index for trend feature
    df = df.reset_index()
    df.rename(columns = {'index': 'Trend'}, inplace = True)
    df['Trend'] += trend_offset # Offset is needed for predictions (offset = number of observations used in model training)

    # Remove dummy dates
    df = df[~df['Date'].isin(dummy_dates)]

    # Remove January column to avoid dummy variable multicollinearity issue
    df.drop(columns = ['Month_January'], inplace = True)

    df.set_index('Date', inplace = True)
    
    return df

X_train = build_features_from_date_series(ts.reset_index()['Date'])

regression_mdl = LinearRegression()
regression_mdl.fit(X_train.to_numpy(), np.log(ts['Sales']).to_numpy())

# Generate dates for the next 12 monthly forecast horizons
test_horizon_dates = pd.Series(pd.date_range(start = X_train.index.max() + pd.DateOffset(months = 1), freq = 'ME', periods = 12))

X_test = build_features_from_date_series(test_horizon_dates, trend_offset = len(X_train))

regression_pred = np.exp(regression_mdl.predict(X_test.to_numpy())) 


### <a id="comparing-model-forecasts">Comparing each model's 12-month forecast</a>

After calculating each model's forecast for the next 12-months we can see they have captured different characteristics of the time series:
- The ETS, ARIMA and regression models all forecasted an upward trend.
- The random walk, ETS and regression models all forecasted the end of year seasonality.
- The ETS and regression models forecasted the increasing seasonality at the end of the year.

The forecasts from the ARIMA model don't appear to be very sensible; they began at a point that was highly influenced by the end of year seasonality and the subsequent forecasts continued to rise from there.



In [163]:
mdl_12month_pred = {
    'Random Walk': list(random_walk_pred),
    'ETS': list(ets_pred),
    'ARIMA': list(arima_pred),
    'Regression': list(regression_pred)
}

mdl_12month_pred = pd.DataFrame(mdl_12month_pred, index = test_horizon_dates)
mdl_12month_pred_long = pd.melt(
    mdl_12month_pred.reset_index(),
    id_vars = 'Date',
    value_vars = mdl_12month_pred.columns,
    var_name = 'Model',
    value_name = 'Sales'
).set_index('Date')

alt.Chart(ts.reset_index()).mark_line().encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = 'Sales'
) + alt.Chart(mdl_12month_pred_long.reset_index()).mark_line().encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = 'Sales',
    color = 'Model'
).properties(
    title = "Actual sales with 12-month forecast for each proposed model"
)

alt.LayerChart(...)

### <a id="model-cross-validation">Model cross validation</a>

ETS models with a seasonal component require at least two years of data for model fitting, so the ETS model could not be validated. The forecasts produced by the ARIMA model also didn't seem very realistic so the ARIMA model was also excluded. This left the random walk and the regression models.

The chosen cross validation procedure is conceptually similar to K-Fold cross validation, while being more suitable for time series data. The train/test split for each fold has been outlined in the figure below. The grey dots indicate the months used for training, and the orange dots indicate test months. The testing was performed on the second forecast horizon (a gap of 1-month was left between training and testing for extra confidence in the training coefficients of the trend line). 



In [167]:
mdl_cv_train_dates = []
mdl_cv_pred_dates = []
mdl_cv_pred = {'Random Walk': [], 'Regression': []}

# Conduct cross validation
# Train random walk and regression models and store the forecast for the test set across 12 splits
# At each split the training set grows by 1 month and testing is always performed on a single month
# The test set is the month following the month that was tested in the previous split

test_horizon = 2 # Number of steps ahead the validation forecast horizon will be
folds = TimeSeriesSplit(n_splits = 12, test_size = 1, gap = test_horizon - 1)
for (i, j) in folds.split(ts):
    ts_cv = ts.iloc[i]
    horizon_date = ts.iloc[j].index

    # Random Walk
    random_walk_mdl_cv = SeasonalNaive(season_length = 12)
    random_walk_mdl_cv.fit(ts_cv['Sales'])
    random_walk_pred_cv = random_walk_mdl_cv.predict(h = test_horizon)['mean'][-1] # Only need the last prediction

    # Regression
    X_train_cv = build_features_from_date_series(ts_cv.reset_index()['Date'])
    regression_mdl_cv = LinearRegression()
    regression_mdl_cv.fit(X_train_cv.to_numpy(), np.log(ts_cv['Sales']).to_numpy())
    X_test_cv = build_features_from_date_series(pd.Series(horizon_date), trend_offset = len(X_train_cv))
    regression_pred_cv = np.exp(regression_mdl_cv.predict(X_test_cv.to_numpy())[0])

    mdl_cv_pred_dates.append(pd.to_datetime(horizon_date.date[0]))
    mdl_cv_pred['Random Walk'].append(random_walk_pred_cv)
    mdl_cv_pred['Regression'].append(regression_pred_cv)

    mdl_cv_train_dates.append(list(ts_cv.reset_index()['Date']))


In [164]:
mdl_cv_train_summary = pd.DataFrame()
mdl_cv_test_summary = pd.DataFrame()

for i, j in enumerate(mdl_cv_train_dates):
    fold_train_dates = pd.DataFrame({'Fold Number': f"{i + 1}", 'Date': j})
    mdl_cv_train_summary = pd.concat([mdl_cv_train_summary, fold_train_dates])

    fold_test_dates = pd.DataFrame({'Fold Number': f"{i + 1}", 'Date': [mdl_cv_pred_dates[i]]})
    mdl_cv_test_summary = pd.concat([mdl_cv_test_summary, fold_test_dates])

# Custom sorting for fold numbers on graph
fold_sort_order = [f"{i + 1}" for i in range(12)]

alt.Chart(mdl_cv_train_summary).mark_point().encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = alt.Y('Fold Number', sort = fold_sort_order)
) + alt.Chart(mdl_cv_test_summary).mark_point(fill = '#FF9933').encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = alt.Y('Fold Number', sort = fold_sort_order)
).properties(
    title = 'An outline of the cross validation procedure (grey = train, orange = test)'
)



alt.LayerChart(...)

### <a id="evaluation-models">Evaluation of the models</a>

The figure below outlines the deviation of each model's forecasts from the actual number of sales. The regression forecasts are higher due to the inclusion of the trend line. Both models underpredicted the end of year seasonality, but the forecasts from the regression model were a lot closer to the true values. The forecasts seem to get more unstable over time, however our sample only contains 26 data points so any further optimisation may lead to overfitting.

The table compares each model's performance; where the regression error measured lower across MSE, RMSE, MAE and MAPE. Using the RMSE as a guideline, the average forecast deviates +/- 97,000 from the actual number of sales.



In [165]:
# Calculate the model error
mdl_cv_pred = pd.DataFrame(mdl_cv_pred, index = mdl_cv_pred_dates)
mdl_cv_true_values = mdl_cv_pred.merge(ts, how = 'inner', left_index = True, right_index = True)['Sales']

mdl_cv_pred_error = mdl_cv_pred.sub(mdl_cv_true_values, axis = 0)
mdl_cv_pred_error.index.name = 'Date'

mdl_cv_pred_error = pd.melt(
    mdl_cv_pred_error.reset_index(),
    id_vars = 'Date',
    value_vars = mdl_cv_pred_error.columns,
    var_name = 'Model',
    value_name = 'Error'
)

mdl_cv_pred_error['Date'] = pd.to_datetime(mdl_cv_pred_error['Date'])

# Dashed line at 0
mdl_line = pd.DataFrame({'Date': mdl_cv_pred_error['Date'].unique(), 'Error': 0})

alt.Chart(mdl_line).mark_line(color = '#808080', strokeDash = [10, 10]).encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = 'Error'
) + alt.Chart(mdl_cv_pred_error).mark_line().encode(
    x = alt.X('Date', timeUnit = 'yearmonth'),
    y = 'Error',
    color = 'Model'
).properties(
    title = "Model error across the testing periods"
)


alt.LayerChart(...)

In [149]:
mdl_names = list(mdl_cv_pred.columns)
mdl_cv_eval_metrics = {'MSE': [], 'RMSE': [], 'MAE': [], 'MAPE': []}

# Calculate evaluation metrics for each model
for i in mdl_names:
    mse = mean_squared_error(mdl_cv_true_values, mdl_cv_pred[i])
    rmse = root_mean_squared_error(mdl_cv_true_values, mdl_cv_pred[i])
    mae = mean_absolute_error(mdl_cv_true_values, mdl_cv_pred[i])
    mape = mean_absolute_percentage_error(mdl_cv_true_values, mdl_cv_pred[i])

    mdl_cv_eval_metrics['MSE'].append(mse)
    mdl_cv_eval_metrics['RMSE'].append(rmse)
    mdl_cv_eval_metrics['MAE'].append(mae)
    mdl_cv_eval_metrics['MAPE'].append(mape)

mdl_cv_eval_summary = pd.DataFrame(mdl_cv_eval_metrics, index = mdl_names)
mdl_cv_eval_summary



,MSE,RMSE,MAE,MAPE
Random Walk,15317587263.083,123764.241,101188.917,0.192
Regression,9411505355.093,97012.913,75794.299,0.153


## <a id="conclusions">Conclusions</a>
***

The regression model performed the strongest through our cross validation procedure, and as a result is our preferred model. The 12-month forecast has been included below.

It is strongly recommended that we repeat this activity again in 12 months time. At that point there will be enough data for the ETS model to be validated and there will hopefully be enough data to detect a more suitable ARIMA model, ideally one with a seasonal component. This additional year of data would also assist us in better understand the time series through proper analysis of the decomposition of seasonality, trend and ACF/PACF plots.



In [168]:
mdl_12month_pred['Regression']

Date
2020-03-31    463990.494
2020-04-30    366756.362
2020-05-31    392122.094
2020-06-30    503815.525
2020-07-31    434125.224
2020-08-31    480966.830
2020-09-30    550936.342
2020-10-31    694149.104
2020-11-30   1351735.847
2020-12-31   1183601.354
2021-01-31    440200.823
2021-02-28    531754.455
Name: Regression, dtype: float64